# Standard Diffusion equation

# Finite Volume

Integral Form

We assume constant c on $Q$ $q =D(x) \nabla c$ is not uniquely defined
on $\partial\Omega$ since we assuse c constant and therefore
discontinous on $\partial Q$. We therefore introduce a Numerical Flux
$q = g(c^+ , c^{-} )$ for example upwind

With
$D(x) = \frac{1}{2+ 1.9 \cos \left( \frac{2 \pi x}{\epsilon} \right)}$
Linear System

``` python
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
epsilon = 0.1 #* np.pi
N = 10
n = 100
h = 1/(N-1)
D = lambda x: 1 / (2+1.9 * np.cos(2 * np.pi* x / epsilon))
x = np.linspace(0,1 ,N)
x_highres = np.linspace(0,1 , 100000)
c = np.zeros(N)
source = np.ones(N) * h
# Boundary
source[0] = 0
source[-1] = 0
plt.plot(x , D(x))
plt.plot(x_highres , D(x_highres))
plt.legend([r"$D$ Sampled on the course grid" , r"$D$"])
plt.title("Diffusion Coefficient")
```

Matrix Assembly

``` python
from scipy.sparse import spdiags
import seaborn as sns
def assemble_matrix(N ,x, h, D ):
    diagp1 = np.zeros(N)
    diagp1[2:] = np.ones(N-2) * -1/h * D(x[1:-1] + 0.5*h)
    diagm1 = np.zeros(N)
    diagm1[:-2] = np.ones(N-2) * -1/h * D(x[1:-1] - 0.5*h)
    diag0 = np.ones(N)
    diag0[1:-1] *= 1/h * (D(x[1:-1]-0.5*h) + D(x[1:-1] + 0.5*h))
    A = spdiags([diagm1 , diag0 , diagp1] , np.array( [-1, 0, 1] ))
    return A
A = assemble_matrix(10 , np.linspace(0,1.,10), h , D_1D)
#plt.spy(A)
sns.heatmap(A.todense())
plt.title("Sparsity Patter of A")
```

``` python
c = sp.sparse.linalg.spsolve(A.tocsr(),source)
plt.plot(c)
plt.title("Course Grid Solution")
```

# Multiscale

In 1D

calculate c integral

``` python
np.sum((c[1:] - c[:-1])/h * -D(x[1:]))
```

``` python
from scipy.sparse.linalg import spsolve
micro_basis = np.zeros(N*n)
T = np.zeros(N)
for i,x_m in enumerate(x):
    xm = np.linspace(x_m , x_m + h , n)
    hm = h/(n-1)
    A_m = assemble_matrix(n, xm , hm )
    fm = np.ones(n) * hm
    fm[0] = 0
    fm[-1] = 1
    phi = spsolve(A_m.tocsr(),fm)
    micro_basis[n * i:n*(i+1)] = phi
    T[i] =   hm * np.sum(((phi[1:] - phi[:-1])/hm)**2 * D(xm[:-1]))
plt.plot(x,T)
plt.xlabel(r"$x$")
plt.ylabel(r"$T(x)$")
plt.title(r"Multiscale Transmission Coeficcients $T$")
```

``` python
diagp1 = np.zeros(N)
diagp1[2:] = np.ones(N-2) * -1* T[1:-1]
diagm1 = np.zeros(N)
diagm1[:-2] = np.ones(N-2) * -1*  T[:-2]
diag0 = np.ones(N)
diag0[1:-1] *=  (T[1:-1] + T[:-2])
A_macro = spdiags([diagm1 , diag0 , diagp1] , np.array( [-1, 0, 1] ))
sns.heatmap(A_macro.todense())
```

``` python
N_fine = n*N
x_fine = np.linspace(0,1 , n*N)
h_fine = 1/(N*n -1)
A_fine = assemble_matrix(N_fine , x_fine , h_fine)
f_fine = np.ones(N*n) * h_fine
f_fine[0] = 0
f_fine[-1] = 0
c_fine = sp.sparse.linalg.spsolve(A_fine.tocsr(),f_fine)
```

``` python
c_macro = sp.sparse.linalg.spsolve(A_macro.tocsr(),source)
c_multi = np.zeros((N-1)* n)
x = np.linspace(0,1,N)
x_multi = np.linspace(0,1 , n*(N-1))
for i in range(len(c_macro)-1):
    t = micro_basis[n*i:n*(i+1)]
    c_multi[n*i:n*(i+1)] = (1-t) * c_macro[i] + t * c_macro[i+1]
plt.plot(x,c)
plt.plot(x,c_macro)
plt.plot(x_multi,c_multi)
plt.plot(x_fine , c_fine)
plt.title("Comparison Of Different Solvers")
plt.xlabel(r"$x$")
plt.ylabel(r"$c(x)$")
plt.legend(["macro" , "multiscale", "multi_fine" , "reference"])
```

# Cleanup

``` python
def assemble_matrix(self)-> None:
   diagp1 = np.zeros(self.N)
   diagp1[2:] =  self._T[1:]
   diagm1 = np.zeros(self.N)
   diagm1[:-2] =  self._T[:-1]
   diag0 = np.ones(self.N)
   diag0[1:-1] = -1 * (self._T[1:] + self._T[:-1])
   self._A = spdiags([diagm1 , diag0 , diagp1] , np.array( [-1, 0, 1] ))
```

``` python
def set_multiscale_transmissions(self, resolution)->NDArray[np.float64]:
   micro_basis = np.zeros((self.N -1)*resolution)
   for i in range(self.N -1):
      micro_fv = FVSolver(resolution , self.D , domain=(self.x[i] , self.x[i+1]))
      micro_fv.set_boundary(bc=(0.,1.))
      micro_fv.assemble_matrix()
      phi = micro_fv.solve()

      micro_basis[resolution * i:resolution*(i+1)] = phi
      hm = micro_fv.h
      self._T[i] = -hm * np.sum(((phi[1:] - phi[:-1])/hm)**2 * self.D(micro_fv.x[:-1]))
   return micro_basis
```

``` python
from typing import Callable
import numpy as np
from scipy.sparse import spdiags
from scipy.sparse.linalg import spsolve
from numpy.typing import NDArray

class FVSolver:
   N : int
   h : np.float64
   x : NDArray[np.float64]
   D : Callable
   f : NDArray[np.float64]
   c : NDArray[np.float64]

   _T : NDArray[np.float64]

   def __init__(self , N :int , D :Callable  , domain=(0.,1.))->None:
       self.h = (domain[1] - domain[0]) / N
       self.N = N
       self.D = D
       self.x = np.linspace(domain[0] , domain[1] , N)
       self._T =  -1/self.h * D((self.x[:-1] + self.x[1:]) * 0.5)
       self.f = self.h* np.ones(N)

<<Assemble Matrix>>


   def set_boundary(self , bc=(0.,0.)):
      self.f[0] = bc[0]
      self.f[-1] = bc[1]


   def solve(self):
      self.c = spsolve(self._A.tocsr() , self.f)
      return self.c

<<Microscale Transmissions>>
```

``` python
from importlib import reload
import src.fvsolver
from src.fvsolver import FVSolver
reload(src.fvsolver)
epsilon = 0.1
D = lambda x: 1 / (2+1.9 * np.cos(2 * np.pi* x / epsilon))
fv = FVSolver(10 ,  D)
fv.assemble_matrix()
fv.set_boundary()
c_course = fv.solve()
plt.plot(c_course)
```

``` python
mb = fv.set_multiscale_transmissions(100)
plt.plot(mb)
```

``` python
fv.assemble_matrix()
c_multi = fv.solve()
plt.plot(c_multi)
```

# 2D

``` python
import scipy as sp
import numpy as np
class FVSolver2D:
   N : int
   M : int
   h_x : np.float64
   h_y : np.float64
   x : NDArray[np.float64]
   y : NDArray[np.float64]
   D : Callable
   f : NDArray[np.float64]
   c : NDArray[np.float64]

   _T_x : NDArray[np.float64]
   _T_y : NDArray[np.float64]

<<Init 2D>>

<<Assemble 2D Matrix>>

   def set_boundary(self , bc=(0.,0. , 0. , 0.)):
      self.f[ 0,1:-1]= bc[0]
      self.f[-1,1:-1]= bc[1]
      self.f[1:-1, 0]= bc[2]
      self.f[1:-1,-1]= bc[3]


   def solve(self):
      self.c = spsolve(self._A.tocsr() , self.f.ravel()).reshape((self.N,self.M))
      return self.c

<<2D Microscale Transmissions>>
```

``` python
def __init__(self ,
             N:int,
             M:int ,
             D :Callable  ,
             domain=np.array([[0.,0.] , [1.,1.]]),
             )->None:
   self.h_x = (domain[1,0] - domain[0,0]) / N
   self.h_y = (domain[1,1] - domain[0,1]) / M
   self.x = np.linspace(domain[0,0] , domain[1,0] , N)
   self.y = np.linspace(domain[0,1] , domain[1,1] , M)
   x_h = self.x[:-1] + 0.5 * self.h_x
   y_h = self.y[:-1] + 0.5 * self.h_y
   halfgrid_x = np.meshgrid(x_h,self.y,indexing="ij")
   halfgrid_y = np.meshgrid(self.x,y_h , indexing="ij")
   self._T_x = -self.h_y/self.h_x * D(halfgrid_x[0] , halfgrid_x[1])
   self._T_y = -self.h_x/self.h_y * D(halfgrid_y[0] , halfgrid_y[1])
   self.N = N
   self.M = M
   self.D = D
   self.f = self.h_x * self.h_y* np.ones((N, M))

```

``` python
def assemble_matrix(self)->None:
    main_diag = np.ones((  self.N,self.M))
    diag_north = np.zeros((self.N,self.M))
    diag_south = np.zeros((self.N,self.M))
    diag_east = np.zeros(( self.N,self.M))
    diag_west = np.zeros(( self.N,self.M))
    main_diag[1:-1,1:-1] =  -1* (self._T_x[:-1,1:-1] + self._T_x[1:,1:-1] + self._T_y[1:-1,:-1] + self._T_y[1:-1,1:])
    main_diag = np.ravel(main_diag)

    diag_north[1:-1,1:-1] =  self._T_y[1:-1,:-1]
    diag_south[1:-1,1:-1] =  self._T_y[1:-1,1:]
    diag_east[1:-1,1:-1] =   self._T_x[1:,1:-1]
    diag_west[1:-1,1:-1] =   self._T_x[:-1,1:-1]
    diag_north = diag_north.ravel()
    diag_south = diag_south.ravel()
    diag_west = diag_west.ravel()
    diag_east = diag_east.ravel()

    A = sp.sparse.spdiags([main_diag , diag_north , diag_south ,  diag_west , diag_east] , [0 , -self.N  , self.N , 1 , -1] , self.N*self.M , self.M*self.N)
    self._A = A.T

```

on a $N \times M$ grid

## Numerical Flux in 2D

The boundary term can then be approximated by

One Dimensionalize the index

plug in Flux Approach with $\Delta_x = \Delta_y = h$

``` python
import os

# Set this before importing NumPy/SciPy
os.environ["OMP_NUM_THREADS"] = "16"       # For MKL/OpenMP
os.environ["OPENBLAS_NUM_THREADS"] = "16"  # For OpenBLAS
os.environ["MKL_NUM_THREADS"] = "16"       # For Intel MKL
os.environ["NUMEXPR_NUM_THREADS"] = "16"   # Just in case

import numpy as np
import scipy

epsilon =0.25
D_1D = lambda x: 1 / (2+1.9 * np.cos(2 * np.pi* x / epsilon))
D = lambda x,y: D_1D(x) * D_1D(y)

alpha = 1.
gamma = 0.001

center = np.array([0.5,0.5])
exp_kernel = lambda r: alpha * np.exp( - r / gamma)
r = 0.2
p = 2.0
thicc = 0.02
R = lambda x,y: np.maximum(0. , np.abs((np.abs(x -center[0])**p + np.abs(y - center[1])**p)**(1/p) - r) - thicc)
D = lambda x,y:   np.maximum(0.0005 , 1. -  exp_kernel(R(x,y)))
D_lin = lambda x,y: x
```

``` python
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
N = 1000
M = 1000
x = np.linspace(0.,1., N)
y= np.linspace(0.,1., M)
grid = np.meshgrid(x,y)
diffusion = D(grid[0] , grid[1])
diffusion = np.reshape(diffusion , (N,M))
sns.heatmap(diffusion)
```

``` python
reload(src.fvsolver)
from src.fvsolver import FVSolver2D
smol_fv = FVSolver2D(10,10,D)
smol_fv.assemble_matrix()
plt.imshow(smol_fv._A.todense())
#plt.spy(A.T, markersize=1)
```

``` python
fv2D = FVSolver2D(N,M,D)
sns.heatmap(fv2D._T_y, cmap="magma")
```

``` python
fv2D = FVSolver2D(N,M,D)
fv2D.assemble_matrix()
fv2D.set_boundary()
c = fv2D.solve()
sns.heatmap(c, cmap="magma")
```

``` python
error =np.linalg.norm(A@c_vec - f)
print(error)
```

``` python
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(grid[0] ,grid[1],c , cmap="magma")
```

# 2D Multiscale

``` python
def set_multiscale_transmissions(self, resolution):
   microscale_basis_x = np.zeros((self._T_x.shape[0] , self._T_x.shape[1] , resolution))
   microscale_basis_y = np.zeros((self._T_y.shape[0] , self._T_y.shape[1] , resolution))
   for i in range(self._T_x.shape[0]):
      for j in range(self._T_x.shape[1]):
         #Do mircroscale x
         D_micro = lambda x: self.D(x, self.y[j])
         fv_micro = FVSolver(resolution , D_micro, domain=(self.x[i] , self.x[i+1]))
         fv_micro.assemble_matrix()
         fv_micro.set_boundary(bc=(0.,1.))
         phi =fv_micro.solve()
         microscale_basis_x[i,j,:] = phi
         self._T_x[i,j] =   -fv_micro.h * self.h_y* np.sum(((phi[1:] - phi[:-1])/fv_micro.h)**2 * D_micro(fv_micro.x[:-1]))

   for i in range(self._T_y.shape[0]):
      for j in range(self._T_y.shape[1]):
         # Do microscale y
         D_micro = lambda y: self.D(self.x[i], y)
         fv_micro = FVSolver(resolution , D_micro, domain=(self.y[j] , self.y[j+1]))
         fv_micro.assemble_matrix()
         fv_micro.set_boundary(bc=(0.,1.))
         phi =fv_micro.solve()
         microscale_basis_y[i,j,:] = phi
         self._T_y[i,j] =   -fv_micro.h * self.h_x  * np.sum(((phi[1:] - phi[:-1])/fv_micro.h)**2 * D_micro(fv_micro.x[:-1]))

   return microscale_basis_x , microscale_basis_y

```

``` python
reload(src.fvsolver)
from src.fvsolver import FVSolver2D
fv2D = FVSolver2D(100,100,D)
mx,my = fv2D.set_multiscale_transmissions(1000)
fv2D.assemble_matrix()
fv2D.set_boundary()
c = fv2D.solve()
sns.heatmap(c, cmap="magma")
```

# Reference Solution

Solution of the 2D Laplace equation:

where $f(x,y) = 2 * (x+y - x^2 - y^2) $ the analytical solution is